In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import time
from scipy.sparse.linalg import svds
from sklearn.metrics import mean_absolute_error
import gzip
import json
import surprise

In [3]:
def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield json.loads(l)

In [4]:
import pandas as pd
import gzip

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

df1 = getDF(r'C:\Users\Kani\Downloads\Electronics_5.json.gz')

In [5]:
df1

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image
0,5.0,67,True,"09 18, 1999",AAP7PPBU72QFM,0151004714,{'Format:': ' Hardcover'},D. C. Carrad,This is the best novel I have read in 2 or 3 y...,A star is born,937612800,NaN
1,3.0,5,True,"10 23, 2013",A2E168DTVGE6SV,0151004714,{'Format:': ' Kindle Edition'},Evy,"Pages and pages of introspection, in the style...",A stream of consciousness novel,1382486400,NaN
2,5.0,4,False,"09 2, 2008",A1ER5AYS3FQ9O3,0151004714,{'Format:': ' Paperback'},Kcorn,This is the kind of novel to read when you hav...,I'm a huge fan of the author and this one did ...,1220313600,NaN
3,5.0,13,False,"09 4, 2000",A1T17LMQABMBN5,0151004714,{'Format:': ' Hardcover'},Caf Girl Writes,What gorgeous language! What an incredible wri...,The most beautiful book I have ever read!,968025600,NaN
4,3.0,8,True,"02 4, 2000",A3QHJ0FXK33OBE,0151004714,{'Format:': ' Hardcover'},W. Shane Schmidt,I was taken in by reviews that compared this b...,A dissenting view--In part.,949622400,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
6739585,4.0,NaN,True,"03 21, 2017",A33MAQA919J2V8,B01HJH40WU,NaN,Kurt Wurm,"These seem like quality USB cables, time will ...",Four Stars,1490054400,NaN
6739586,4.0,NaN,True,"01 9, 2017",A1AKHSCPD1BHM4,B01HJH40WU,NaN,C.L Momof3,"Works great, love the longer cord. As with any...",Nice long cord,1483920000,NaN
6739587,5.0,2,True,"12 1, 2016",A2HUZO7MQAY5I2,B01HJH40WU,NaN,michael clontz,"Ok here is an odd thing that happened to me, I...",Not the correct product as linked in the sale.,1480550400,NaN
6739588,5.0,2,True,"11 29, 2016",AJJ7VX2L91X2W,B01HJH40WU,NaN,Faith,Works well.,Five Stars,1480377600,NaN


In [6]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split 

subset_80, subset_20 = train_test_split(df1, test_size=0.035, stratify=df1['overall'])

In [7]:
subset_20

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image
1425280,2.0,NaN,True,"01 24, 2015",AMFPAS8VMIDGJ,B002B3YCQM,{'Package Type:': ' Standard Packaging'},Crewtx,I've stopped counting the number of mice I've ...,I've stopped counting the number of mice I've ...,1422057600,NaN
4253927,4.0,2,True,"12 3, 2017",ABID49T47BLBF,B00JXXXOS0,NaN,Donald L. DeMent,I've only had it 1 day but it worked for me pl...,Worked for playing local (R 1) DVDs and the Au...,1512259200,NaN
3395176,5.0,NaN,True,"09 24, 2014",A2IQGDEA5FONCT,B00CSQYVWM,{'Style:': ' Skeleton Housing'},JT,perfect for indoors and automobile use.,Five Stars,1411516800,NaN
1421371,4.0,2,True,"12 5, 2012",A29GIVFYIGNM79,B002AF5184,NaN,Music Is Everything,This is an inexpensive but effective case. It...,"Nice, Inexpensive, Easy-Access Case",1354665600,NaN
2768360,5.0,NaN,True,"01 5, 2016",A1MPC88C5RBO91,B008LTBM6S,{'Capacity:': ' 8GB Kit (4GBx2)'},Max Branvall,I got the 4x2 8gig set for mid 2012 MacBook Pr...,Exactly as advertised,1451952000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2741767,5.0,2,True,"02 6, 2013",A28FXEB3VIG8DU,B008DBI5RI,NaN,Alexander Seifert,"This was a Christmas gift for my 6 year old, w...",very good product,1360108800,NaN
3967102,5.0,NaN,False,"06 4, 2016",A1MGUMKNMC5DL5,B00HNDCWQM,{'Color:': ' 4.5 Feet'},Clyde Allen Gullett,I had lost all of these cables in moving. I st...,Great cables,1464998400,NaN
1020058,5.0,NaN,True,"03 5, 2015",A237SN14B2I5AX,B0014KQ4NM,{'Length:': ' 1-Pack'},Colleen P Quigley,Works perfectly!,Five Stars,1425513600,NaN
3661069,3.0,NaN,True,"11 1, 2015",A3VUQ1W0WOQAX4,B00EPCNB96,{'Style:': ' Receiver'},FAMILY OKAT,"Works so well, was surprised - once we got the...","Works, but cord too short",1446336000,NaN


In [8]:
subset_20.to_csv('amazon_electronic_subset.csv')

In [10]:
import pandas as pd
import gzip

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

df2 = getDF(r'C:\Users\Kani\Downloads\meta_Electronics.json.gz')

In [11]:
df2

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
0,"[Electronics, Camera &amp; Photo, Video Survei...",,[The following camera brands and models have b...,,Genuine Geovision 1 Channel 3rd Party NVR IP S...,[],,GeoVision,"[Genuine Geovision 1 Channel NVR IP Software, ...","[>#3,092 in Tools &amp; Home Improvement &gt; ...",[],Camera &amp; Photo,,"January 28, 2014",$65.00,0011300000,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
1,"[Electronics, Camera &amp; Photo]",,[This second edition of the Handbook of Astron...,,"Books ""Handbook of Astronomical Image Processi...",[0999470906],,33 Books Co.,[Detailed chapters cover these fundamental top...,"[>#55,933 in Camera &amp; Photo (See Top 100 i...","[0943396670, 1138055360, 0999470906]",Camera &amp; Photo,,"June 17, 2003",,0043396828,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
2,"[Electronics, eBook Readers &amp; Accessories,...",,[A zesty tale. (Publishers Weekly)<br /><br />...,,One Hot Summer,"[0425167798, 039914157X]",,Visit Amazon's Carolina Garcia Aguilera Page,[],"3,105,177 in Books (",[],Books,,,$11.49,0060009810,[],[],NaN
3,"[Electronics, eBook Readers & Accessories, eBo...",,[],,Hurray for Hattie Rabbit: Story and pictures (...,"[0060219521, 0060219580, 0060219394]",,Visit Amazon's Dick Gackenbach Page,[],"2,024,298 in Books (","[0060219521, 0060219475, 0060219394]",Books,,,.a-section.a-spacing-mini{margin-bottom:6px!im...,0060219602,[],[],NaN
4,"[Electronics, eBook Readers & Accessories, eBo...",,[&#8220;sex.lies.murder.fame. is brillllli&#82...,,sex.lies.murder.fame.: A Novel,[],,Visit Amazon's Lolita Files Page,[],"3,778,828 in Books (",[],Books,,,$13.95,0060786817,[],[],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
786440,"[Electronics, Portable Audio & Video, MP3 & MP...","class=""a-keyvalue prodDetTable"" role=""present...",[<b>Specification: </b><br>Item Type: Cassette...,,Tukzer Cassette Player Portable Tape Player Ca...,[],,Tukzer,"[With playback, can check the recorded music o...",[>#217 in Electronics > Accessories & Supplies...,[],Portable Audio & Accessories,,"September 18, 2014",,B01HJGOOMW,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,{}
786441,"[Electronics, Computers & Accessories, Laptop ...",,[<br />Cobiter AC adapters are made with the h...,,Cobiter AC Adapter Battery Charger For IBM LEN...,[],,Cobiter,[Cobiter product has 30 days Money-Back Guaran...,"[>#856,399 in Electronics (See Top 100 in Elec...",[],Computers,,"June 25, 2016",,B01HJH3XLE,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,{}
786442,"[Electronics, Portable Audio & Video, CB & Two...","class=""a-keyvalue prodDetTable"" role=""present...",[</b>Specifications:<br><br> Original Li-ion B...,,Baofeng Original Li-ion Battery Charger Deskto...,[],,COOLBOY,[Baofeng Li-ion Battery Charger Desktop Charge...,"[>#1,882 in Electronics > Portable Audio & Vid...",[],GPS & Navigation,,"June 25, 2016",,B01HJH6CEY,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,{}
786443,"[Electronics, Computers & Accessories, Compute...",,[],,Jricoo 3pack 6ft Micro USB to USB Cable 2.0 6f...,[],,Jricoo,[],"[>#9,596 in Industrial & Scientific > Industri...",[],Cell Phones & Accessories,,,,B01HJH40WU,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,{}


In [12]:
df_inner = pd.merge(subset_20,df2, on='asin', how='inner')

df_inner

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,...,feature,rank,also_view,main_cat,similar_item,date,price,imageURL,imageURLHighRes,details
0,2.0,NaN,True,"01 24, 2015",AMFPAS8VMIDGJ,B002B3YCQM,{'Package Type:': ' Standard Packaging'},Crewtx,I've stopped counting the number of mice I've ...,I've stopped counting the number of mice I've ...,...,[ERGONOMIC SHAPE - Ergonomically shaped design...,[>#50 in Computers & Accessories > Computer Ac...,[],All Electronics,"class=""a-bordered a-horizontal-stripes a-spa...","June 1, 2009",$2.54,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,{}
1,4.0,NaN,True,"03 26, 2016",A1UVU5YY6B6KQM,B002B3YCQM,{'Package Type:': ' Standard Packaging'},Oohla,this mouse worked for 1 month and 10 days. To...,Worked for under 2 months - Logitech was happy...,...,[ERGONOMIC SHAPE - Ergonomically shaped design...,[>#50 in Computers & Accessories > Computer Ac...,[],All Electronics,"class=""a-bordered a-horizontal-stripes a-spa...","June 1, 2009",$2.54,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,{}
2,5.0,NaN,True,"11 28, 2015",A2G8K3GDE6R2Z5,B002B3YCQM,{'Package Type:': ' Standard Packaging'},Rick Strahl,I bought a tethered mouse after buying a new 4...,Love the Scroll Wheel on this Mouse,...,[ERGONOMIC SHAPE - Ergonomically shaped design...,[>#50 in Computers & Accessories > Computer Ac...,[],All Electronics,"class=""a-bordered a-horizontal-stripes a-spa...","June 1, 2009",$2.54,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,{}
3,3.0,NaN,True,"01 24, 2014",A1N499QUWCGE4N,B002B3YCQM,{'Package Type:': ' Standard Packaging'},Kev,The left button has started misbehaving after ...,worn out after 3 years but ordering a new one,...,[ERGONOMIC SHAPE - Ergonomically shaped design...,[>#50 in Computers & Accessories > Computer Ac...,[],All Electronics,"class=""a-bordered a-horizontal-stripes a-spa...","June 1, 2009",$2.54,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,{}
4,4.0,NaN,True,"01 3, 2014",A1OV75M25ENAND,B002B3YCQM,{'Package Type:': ' Standard Packaging'},Todd S.,Only one problem with this mouse. Those confou...,A really comfortable mouse to use and just the...,...,[ERGONOMIC SHAPE - Ergonomically shaped design...,[>#50 in Computers & Accessories > Computer Ac...,[],All Electronics,"class=""a-bordered a-horizontal-stripes a-spa...","June 1, 2009",$2.54,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,{}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246840,5.0,NaN,True,"11 7, 2015",ADDOTT8YOT0NS,B0048ABG6A,NaN,P. Cox,"The adapter works great. However, I haven't fi...",Fotodiox Lens Adapter,...,[Mounts Canon FD & FL 35mm SLR lenses to Sony ...,[>#946 in Camera & Photo > Camera & Photo Acce...,"[B003Y302CG, B00LECYIWE, B003EB0AY8, B00EDBT44...",Camera & Photo,"class=""a-bordered a-horizontal-stripes a-spa...","October 22, 2010",$4.14,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,{}
246841,5.0,6,True,"08 9, 2011",AKH6N84CBOBJ6,B004WI867K,NaN,Eric Mars,For the price I figured I'd take a chance on t...,Attractive & Well Made,...,"[For Macbook air 11.6"" A1370 only, Made of PU/...","[>#5,651 in Computers & Accessories > Laptop A...",[],Computers,"class=""a-bordered a-horizontal-stripes a-spa...","April 7, 2011",,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,{}
246842,5.0,NaN,True,"03 26, 2018",A2J0CYSHN37GA1,B01E5ITE2W,NaN,AmazonCustomer,Works great for capturing old VHS tapes and co...,Works well!,...,[It can capture High-quality video and audio f...,[>#5 in Computers & Accessories > Computer Com...,"[B0029U2YSA, B06X42H9VZ, B000VM60I8, B07H287NR...",Computers,"class=""

In [15]:
df1.columns

Index(['overall', 'vote', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'style', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime',
       'image'],
      dtype='object')

In [14]:
df_inner.columns

Index(['overall', 'vote', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'style', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime',
       'image', 'category', 'tech1', 'description', 'fit', 'title', 'also_buy',
       'tech2', 'brand', 'feature', 'rank', 'also_view', 'main_cat',
       'similar_item', 'date', 'price', 'imageURL', 'imageURLHighRes',
       'details'],
      dtype='object')

In [17]:
from copy import deepcopy
df_meta = deepcopy(df_inner)
df_meta= df_meta.drop(['overall', 'vote', 'verified', 'reviewTime', 'reviewerID',
       'style', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime',
       'image'],axis=1)

In [23]:
df_meta=df_meta[['asin','category','title']]

In [24]:
df_meta

,asin,category,title
0,B002B3YCQM,"[Electronics, Computers & Accessories, Compute...",Logitech M500 Corded Mouse &ndash; Wired USB M...
1,B002B3YCQM,"[Electronics, Computers & Accessories, Compute...",Logitech M500 Corded Mouse &ndash; Wired USB M...
2,B002B3YCQM,"[Electronics, Computers & Accessories, Compute...",Logitech M500 Corded Mouse &ndash; Wired USB M...
3,B002B3YCQM,"[Electronics, Computers & Accessories, Compute...",Logitech M500 Corded Mouse &ndash; Wired USB M...
4,B002B3YCQM,"[Electronics, Computers & Accessories, Compute...",Logitech M500 Corded Mouse &ndash; Wired USB M...
...,...,...,...
246840,B0048ABG6A,"[Electronics, Camera & Photo, Accessories, Tri...",Fotodiox Pro Lens Mount Adapter - Canon FD &am...
246841,B004WI867K,"[Electronics, Computers & Accessories, Laptop ...",Cosmos &reg; Brown PU/synthetic/faux leather 1...
246842,B01E5ITE2W,"[Electronics, Computers & Accessories, Compute...",REDGO Video Audio VHS VCR USB Video Capture Ca...
246843,B008XI59K2,"[Electronics, Computers & Accessories, Laptop ...",HP 2000 2000-2C29WM 2000-2B19WM 693711-001 677...


In [25]:
df_meta=df_meta.drop_duplicates(subset=['asin','title'],keep='last')

In [26]:
df_meta

,asin,category,title
24,B002B3YCQM,"[Electronics, Computers & Accessories, Compute...",Logitech M500 Corded Mouse &ndash; Wired USB M...
25,B00JXXXOS0,"[Electronics, Television & Video, DVD Players ...",Panasonic All Multi Region Code Zone Free PAL/...
35,B00CSQYVWM,"[Electronics, Camera & Photo, Underwater Photo...",GoPro HERO3 Skeleton Housing
44,B002AF5184,"[Electronics, Portable Audio & Video, MP3 & MP...",CrazyOnDigital Premium Black Leather Case Appl...
115,B008LTBM6S,"[Electronics, Computers & Accessories, Compute...",Crucial 4GB Single DDR3/DDR3L 1600 MT/s (PC3-1...
...,...,...,...
246840,B0048ABG6A,"[Electronics, Camera & Photo, Accessories, Tri...",Fotodiox Pro Lens Mount Adapter - Canon FD &am...
246841,B004WI867K,"[Electronics, Computers & Accessories, Laptop ...",Cosmos &reg; Brown PU/synthetic/faux leather 1...
246842,B01E5ITE2W,"[Electronics, Computers & Accessories, Compute...",REDGO Video Audio VHS VCR USB Video Capture Ca...
246843,B008XI59K2,"[Electronics, Computers & Accessories, Laptop ...",HP 2000 2000-2C29WM 2000-2B19WM 693711-001 677...


In [27]:
df_meta.to_csv('meta_amazon_electronic_subset.csv')

In [28]:
df_inner = pd.merge(subset_20,df_meta, on='asin', how='inner')

df_inner

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image,category,title
0,2.0,NaN,True,"01 24, 2015",AMFPAS8VMIDGJ,B002B3YCQM,{'Package Type:': ' Standard Packaging'},Crewtx,I've stopped counting the number of mice I've ...,I've stopped counting the number of mice I've ...,1422057600,NaN,"[Electronics, Computers & Accessories, Compute...",Logitech M500 Corded Mouse &ndash; Wired USB M...
1,4.0,NaN,True,"03 26, 2016",A1UVU5YY6B6KQM,B002B3YCQM,{'Package Type:': ' Standard Packaging'},Oohla,this mouse worked for 1 month and 10 days. To...,Worked for under 2 months - Logitech was happy...,1458950400,NaN,"[Electronics, Computers & Accessories, Compute...",Logitech M500 Corded Mouse &ndash; Wired USB M...
2,5.0,NaN,True,"11 28, 2015",A2G8K3GDE6R2Z5,B002B3YCQM,{'Package Type:': ' Standard Packaging'},Rick Strahl,I bought a tethered mouse after buying a new 4...,Love the Scroll Wheel on this Mouse,1448668800,NaN,"[Electronics, Computers & Accessories, Compute...",Logitech M500 Corded Mouse &ndash; Wired USB M...
3,3.0,NaN,True,"01 24, 2014",A1N499QUWCGE4N,B002B3YCQM,{'Package Type:': ' Standard Packaging'},Kev,The left button has started misbehaving after ...,worn out after 3 years but ordering a new one,1390521600,NaN,"[Electronics, Computers & Accessories, Compute...",Logitech M500 Corded Mouse &ndash; Wired USB M...
4,4.0,NaN,True,"01 3, 2014",A1OV75M25ENAND,B002B3YCQM,{'Package Type:': ' Standard Packaging'},Todd S.,Only one problem with this mouse. Those confou...,A really comfortable mouse to use and just the...,1388707200,NaN,"[Electronics, Computers & Accessories, Compute...",Logitech M500 Corded Mouse &ndash; Wired USB M...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235615,5.0,NaN,True,"11 7, 2015",ADDOTT8YOT0NS,B0048ABG6A,NaN,P. Cox,"The adapter works great. However, I haven't fi...",Fotodiox Lens Adapter,1446854400,NaN,"[Electronics, Camera & Photo, Accessories, Tri...",Fotodiox Pro Lens Mount Adapter - Canon FD &am...
235616,5.0,6,True,"08 9, 2011",AKH6N84CBOBJ6,B004WI867K,NaN,Eric Mars,For the price I figured I'd take a chance on t...,Attractive & Well Made,1312848000,NaN,"[Electronics, Computers & Accessories, Laptop ...",Cosmos &reg; Brown PU/synthetic/faux leather 1...
235617,5.0,NaN,True,"03 26, 2018",A2J0CYSHN37GA1,B01E5ITE2W,NaN,AmazonCustomer,Works great for capturing old VHS tapes and co...,Works well!,1522022400,NaN,"[Electronics, Computers & Accessories, Compute...",REDGO Video Audio VHS VCR USB Video Capture Ca...
235618,5.0,NaN,True,"09 14, 2014",AVW3SN671V6Q,B008XI59K2,NaN,AW,"Worked great, great price.",Perfect,1410652800,NaN,"[Electronics, Computers & Accessories, Laptop ...",HP 2000 2000-2C29WM 2000-2B19WM 693711-001 677...


In [29]:
l=df_inner['category']
import numpy as np
gfg = np.unique(l)
  
print(gfg)

[list(['Electronics', 'Accessories & Supplies'])
 list(['Electronics', 'Accessories & Supplies', '</span></span></span>', 'Can Accommodate Up to Three Different Gaming Consoles at The Same Time', 'Specially Designed Hooks Keep Controllers Safely Off The Floor', 'Optional Leveling Feet or Lockable Casters Provide Flexibility of Use', 'Pull-out Drawer Provides Unobstructed View of the Titles Spine Up', 'Part of a Modular System It Can Be Expanded Without Limit', 'Organizes and stores your gaming gear', 'Swiveling wheels for easy maneuverability'])
 list(['Electronics', 'Accessories & Supplies', 'Audio & Video Accessories'])
 ...
 list(['Electronics', 'eBook Readers &amp; Accessories', 'Power Cables'])
 list(['Electronics', 'eBook Readers &amp; Accessories', 'Reading Lights'])
 list(['Electronics', 'eBook Readers &amp; Accessories', 'eBook Readers'])]
